# Production Module in Partial Equilibrium

In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb
os.chdir(main)
%run stdPlotting.ipynb
os.chdir(d['py'])
from gmsPython import nestingTree
from valueShares import nestedShares
from mProduction import NestedCES, NestedCES_PC

The file _gams_py_gdb0.gdx is still active and was not deleted.


## 1. Data and nesting structure

Load data:

In [2]:
t0 = 2019 # set baseline year
name = 'vLarge' # specify name of the CGE version we are currently working on
db = GpyDB(os.path.join(d['data'], f'{name}_{t0}'), name = '_'.join([name, str(t0), 'SimpleEOP']), ws = d['work']) # load and rename database 
dbIO = db.copy() # create copy without adjustments made along the way
ws = db.ws # run everything from the same ws

Subset database to only include production sector variables:

In [3]:
# AggDB.subsetDB(db, db('s_p'))

The production module is initialized with a nesting tree and value shares to identify the $\mu$-share parameters of the nesting structure.

In [4]:
nest = nestingTree.Tree('P', tree = db('nestProduction').to_list(), f = 'CES')
nestAgg = nestingTree.AggTree(name = 'P', trees = {t.name: t for t in [nest]}, ws = ws)
nestAgg(namespace = {str(n)+'_input':n for n in db('n')})

Create value shares:

In [5]:
v = nestedShares(nestAgg, ws = ws)
db_p = v(dbIO)

Use share parameters and values to get initial values for intermediates:

In [6]:
db.aom(db_p('mu').xs(t0), name = 'mu')
db.aom(adj.rc_pd(db_p('vD'), nest.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

Use static user cost as initial guess for price on durables (if no other has been provided):

In [7]:
db.aom(db('pD_dur'), name = 'pD', priority = 'first') # pD_dur includes an estimate of the long-run user-cost of capital

## 2. Small stuff

*Clean up database a bit (this is not necessary, but it removes some variables that are not ultimately used in the model):*

In [8]:
[db.series.__delitem__(k) for k in ('vD','vTax', 'vD_dur','vD_depr','vD_inv', 'vS', 'pD_dur') if k in db.symbols];

For variables that are defined over $t$, but where we do not yet have an initial value for all $t$, extrapolate from data:

*Note: This forces extrapolation of all variables defined over $t$ - if it is important that some variables are not extrapolated, they should be removed from this statement.*

In [9]:
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

## 3. Initialize and calibrate model

Initialize production module from the nesting tree. Specify that we are solving in partial equilibrium, that we want to draw on the method ```initFirmValue``` (from ```gamsProduction.py```) to get some of the initial values, and that we want to use the lump sum tax to calibrate total transfers to the government (choose between tauLump, tauS, tauD):

In [10]:
m = NestedCES(nestAgg, partial = True, initFromGms = 'initFirmValue', taxInstr = 'tauLump')
m.initStuff(db = db)

Use $j$-terms solution approach for calibration:

In [11]:
soldb = m.jSolve(25, state = 'C', ϕ = .75)

## 4. Use calibration solution as initial data, test baseline solution

Write solution to the main database again:

In [12]:
[m.db.__setitem__(k, soldb[k]) for k in m.db.getTypes(['var']) if k in soldb.symbols]; # use solution database
m.db.mergeInternal()

Remove init methods (now we want to use values directly from database):

In [13]:
m.initFromGms = None

Test baseline solution:

In [14]:
baseSol = m.solve(state = 'B')

## 5. Test version of "perfect competition"

In [15]:
mpc = NestedCES_PC(nestAgg, partial = True, taxInstr = 'tauLump')
mpc.initStuff(db = db)
soldb = mpc.jSolve(10, state = 'C')

C:\Users\sxj477\Anaconda3\envs\GAMS46\Lib\site-packages\pyDatabases\gpyDB\database.py:525: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return ss.combine_first(s)
C:\Users\sxj477\Anaconda3\envs\GAMS46\Lib\site-packages\pyDatabases\gpyDB\database.py:525: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return ss.combine_first(s)
C:\Users\sxj477\Anaconda3\envs\GAMS46\Lib\site-packages\pyDatabases\gpyDB\database.py:525: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty i